In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
 
# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model
 
# run the test harness for evaluating a model
def run_test_harness():
# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow_from_directory('/content/drive/MyDrive/train/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	# fit model
	model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=10, verbose=0)
	# save model
	model.save('final_model.h5')
 
# entry point, run the test harness
run_test_harness()

In [81]:
import numpy as np
from PIL import Image
import os
from google.colab.patches import cv2_imshow
import cv2

def convert_to_img(path):
  f = open(path, mode='r',encoding='utf-8')
  image = np.fromfile(f, dtype=np.ubyte)
  filesize = image.shape[0]
  width = 224
  rem = filesize%width
  if rem != 0:
    image = image[:-rem]
  height = int(image.shape[0]/width)      #height based on file size
  image = image.reshape(height,width)
  #im = Image.fromarray(image)
  return image

In [109]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

  # load and prepare the image
def load_image(path,flag_of_img=False):
# load the image
  if (flag_of_img==False):
    image=convert_to_img(path)
    #print(type(image))
    #image = np.array(image)
    #print(type(image))
    new_image = np.copy(image)
    print(type(new_image))
    #new_image = new_image.reshape(-1)
    new_image = np.resize(new_image, (224,224,3))
    new_image= np.expand_dims(new_image, axis= 0)

    #new_image = new_image.reshape(-1,1)
    #new_image=new_image.resize((224,224))
    #image = np.array(image)
    
    print(type(new_image))
    print(new_image.shape)
 
    img = new_image.reshape(1,224,224,3)
    #image= np.expand_dims(np.array(image, dtype= float)/255, axis= 0)
    #new_image = new_image.reshape(-1,1)
  if (flag_of_img):
    filename = path
    image=load_img(filename,target_size=(224,224))
  # convert to array
    img=img_to_array(image)
	# reshape into a single sample with 3 channels
    img = img.reshape(1, 224, 224, 3)
  # center pixel data
  img = img.astype('float32')
  img = img - [123.68, 116.779, 103.939]
  return img
 
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('/content/drive/MyDrive/i170217-Q3 - Afaq Asif.cpp',False)
	# load model
	model = load_model('/content/drive/MyDrive/complete_classifier.h5')
	# predict the class
	result = model.predict(img)
	print(result[0])
 
# entry point, run the example
run_example()

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1, 224, 224, 3)
[0.08429724]


In [ ]:
#model = load_model('final_model.h5')
model = load_model('/content/drive/MyDrive/complete_classifier.h5')
#model.save('/content/drive/MyDrive/complete_classifier.h5')

NameError: ignored